<a href="https://colab.research.google.com/github/okioma/Mugo/blob/main/indices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import Google earth engine module
import ee
import geemap


# Authenticate the Google earth engine with google account
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=areEhfPhYXqlVb1BIOhvZ9m-5HHWpKL99fmfSSHEgQQ&tc=VU0t-vVaC-f8dbZmeelfURbBHBoD0yWLY7vAWB7ZJ2E&cc=q8YlOryLDsOn-aGLgaK-U5vgQ-2VhoSUvBXxAWADc8s

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJrszyIDMprDGQ6hcJa9WFLQ-0vu9UArkxBXVyUHgjVvauVg-vgYjKY

Successfully saved authorization token.


In [ ]:
pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 26.4 MB/s 
     |████████████████████████████████| 1.2 MB 50.5 MB/s 
     |████████████████████████████████| 1.3 MB 62.7 MB/s 
     |████████████████████████████████| 3.0 MB 38.2 MB/s 
     |████████████████████████████████| 130 kB 78.1 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 46 kB 4.6 MB/s 
     |████████████████████████████████| 99 kB 10.5 MB/s 
     |████████████████████████████████| 3.7 MB 61.1 MB/s 
     |████████████████████████████████| 198 kB 84.3 MB/s 
     |████████████████████████████████| 8.8 MB 40.5 MB/s 
     |████████████████████████████████| 98 kB 9.0 MB/s 
     |████████████████████████████████| 128 kB 77.9 MB/s 
     |████████████████████████████████| 1.6 MB 68.3 MB/s 
     |████████████████████████████████| 423 kB 74.0 MB/s 
     |████████████████████████████████

In [ ]:
Map=geemap.Map()
Map.add_basemap('NatGeoWorldMap')  # Add Google Map
my_map = geemap.Map(location=[37.9062, 0.0236], zoom_start=10)

In [ ]:
#name of bands
inBands = ee.List(['QA60','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']);
outBands = ee.List(['QA60','blue','green','red','re1','re2','re3','nir','re4','waterVapor','cirrus','swir1','swir2']);
 
CloudCoverMax = 20

#function to get the data
def importData(studyArea,startDate,endDate):
    # Get Sentinel-2 data
    s2s =(ee.ImageCollection('COPERNICUS/S2').filterDate(startDate,endDate).filterBounds(studyArea).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',CloudCoverMax)).filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT',CloudCoverMax)))
    
    #sentinel bands are in scale of 0.0001
    def scaleBands(img):
        prop = img.toDictionary()
        t = (img.select(['QA60','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']).divide(10000))
        t = (t.addBands(img.select(['QA60'])).set(prop).copyProperties(img,['system:time_start','system:footprint']))
             
        return ee.Image(t)
    
    
    s2s = s2s.map(scaleBands)
    s2s = s2s.select(inBands,outBands)
    
    return s2s

In [ ]:
startyear = 2019
endyear = 2019

startDate = ee.Date.fromYMD(startyear,1,1)
endDate = ee.Date.fromYMD(endyear,12,31)

#gee assets to get the study area
studyArea = ee.FeatureCollection('users/mautiharonokioma/transzoia')

print("Getting images") 
s2 = importData(studyArea, startDate,endDate)
visParams = { 'min': 0.0,
  'max': 0.35,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]}

S2 = s2.median().clip(studyArea)


Getting images


In [ ]:
# get indexes
def getNDVI(image):
    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['nir','red']).rename("ndvi")
    image = image.addBands(ndvi)

    return(image)



In [ ]:
#get Indexes
print("getting indexes")
s2 = s2.map(getNDVI)
SentinelNdvi = s2.select('nd')

Map.addLayer(SentinelNdvi, visParams, "Sentinel image")
Map

getting indexes


EEException: ignored